## Background Exploration
In the previous iteration of the paper, bucket sizes were chosen to include a balanced number from each class from the training set.

With cross-validation, it no longer seems ideal to pick the buckets based on that. Here, we examine the distributions.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
if 'notebooks' not in os.listdir():
    # want to go up one level
    os.chdir('..') 
    
if os.environ.get("PYTHONHASHSEED") != "0":
    raise Exception("You must set PYTHONHASHSEED=0 when starting the Jupyter server to get reproducible results.")

from collections import Counter, defaultdict
    
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, mean

import util
from evaluation.surface_level_probing import *
from util.build_student_vectors import get_student_vectors
from util.enums import RepresentationLevel, VectorMethod
from util.prep_location_embeddings import get_location_embeddings, get_multiple_location_embeddings

## Load all of the embeddings that will be used for these tasks
### These will be converte to sequence representations

In [ ]:
# load embeddings to test
whitelist = 'data/building_whitelists/location_twitter_reddit_mention_campus_info.whitelist'

onehot, _ = get_location_embeddings(
    onehot=True, seed=884, use_whitelist=whitelist)

loc2v_embedding_list, _ = get_multiple_location_embeddings(
    vector_size=25, window_size=5, epochs=5, skip_gram=True, negative=20, 
    hour_chunks=True, use_whitelist=whitelist)

twitter_geo_embeddings, _ = get_location_embeddings(
    use_whitelist=whitelist, only_text_embeddings=True,
    text_embedding_methods=['twitter_geo.tfidf_pretrained.glove_twitter'])

reddit_embeddings, _ = get_location_embeddings(
    use_whitelist=whitelist, only_text_embeddings=True,
    text_embedding_methods=['reddit_mention.tfidf_pretrained.glove_twitter'])

campus_info_embeddings, _ = get_location_embeddings(
    use_whitelist=whitelist, only_text_embeddings=True,
    text_embedding_methods=['campus_info.tfidf_pretrained.glove_twitter'])

loc2v_reddit_embedding_list, _ = get_multiple_location_embeddings(
    vector_size=25, window_size=5, epochs=5, skip_gram=True, negative=20,
    use_whitelist=whitelist, hour_chunks=True,
    text_embedding_methods=['reddit_mention.tfidf_pretrained.glove_twitter'])

reddit_retrofit_embeddings_weighted, _ = get_location_embeddings(
    use_whitelist=whitelist, only_text_embeddings=True, retrofit_location_graph_weighted=True,
    text_embedding_methods=['reddit_mention.tfidf_pretrained.glove_twitter'])

method_to_emb = {
    "One-Hot Avg": onehot,
    "Loc2V": loc2v_embedding_list,
    "Twitter": twitter_geo_embeddings,
    "Reddit": reddit_embeddings,
    "Campus-Website": campus_info_embeddings,
    "Loc2V-Reddit": loc2v_reddit_embedding_list,
    "Reddit-Retrofit": reddit_retrofit_embeddings_weighted,
}

with open(whitelist, "r") as f:
    vocab = f.read().splitlines()
    
# confirm perfect overlap of keys
for i in range(10):
    assert(onehot.keys() == loc2v_embedding_list[i].keys() == 
           twitter_geo_embeddings.keys() == reddit_embeddings.keys() == campus_info_embeddings.keys() == 
           reddit_retrofit_embeddings_weighted.keys())

In [ ]:
method_to_vectors_and_sequences = {}
for method, emb in method_to_emb.items():
    if type(emb) == list:
        vec_seq_list = []
        for ind_emb in emb:
            vec_seq_list.append(prep_embeddings(ind_emb))
        method_to_vectors_and_sequences[method] = vec_seq_list
    else:
        method_to_vectors_and_sequences[method] = [prep_embeddings(emb)]

## Location Presence Probing Task

In [ ]:
method_to_f1_by_build = {}
for method, vec_seq_list in method_to_vectors_and_sequences.items():
    print(method)
    _, _, _, _, _, f1_by_build = location_presence_multiple(vec_seq_list)
    method_to_f1_by_build[method] = f1_by_build

## Functionality Presence Probing Task

In [ ]:
method_to_f1_by_func = {}
for method, vec_seq_list in method_to_vectors_and_sequences.items():
    print(method)
    _, _, _, _, _, f1_by_func = functionality_presence_multiple(vec_seq_list)
    method_to_f1_by_func[method] = f1_by_func